# RAG (Retrieval Augmented Generation) with LangChain

This notebook demonstrates how to implement a RAG system using LangChain. We'll cover:

1. Loading and processing documents
2. Creating vector embeddings
3. Setting up a vector store
4. Implementing retrieval
5. Generating responses using retrieved context

Let's get started!


In [1]:
# Install required packages
%pip install langchain langchain-openai chromadb tiktoken

Note: you may need to restart the kernel to use updated packages.


In [2]:
# Import necessary libraries
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_community.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
import os

# Load environment variables from .env file
from dotenv import load_dotenv
load_dotenv()  # This will load variables from a .env file into the environment

# Get your OpenAI API key from the environment
openai_api_key = os.getenv("OPENAI_API_KEY")
if not openai_api_key:
    raise ValueError("OPENAI_API_KEY not found in environment. Please set it in your .env file.")


## 1. Loading and Processing Documents

First, let's create a sample document and load it using LangChain's document loader. Then, we'll split it into smaller chunks for better processing.


In [3]:
# Create a sample document
with open("sample_doc.txt", "w") as f:
    f.write("""
Artificial Intelligence (AI) is revolutionizing various industries. Machine learning, a subset of AI, 
enables computers to learn from data without being explicitly programmed. Deep learning, a type of 
machine learning, uses neural networks with multiple layers to process complex patterns.

Natural Language Processing (NLP) is another important field in AI. It focuses on enabling computers 
to understand, interpret, and generate human language. Applications of NLP include machine translation, 
sentiment analysis, and chatbots.

Computer Vision is the field of AI that enables computers to understand and process visual information 
from the world. It has applications in facial recognition, autonomous vehicles, and medical imaging.
""")

# Load the document
loader = TextLoader("sample_doc.txt")
documents = loader.load()

# Split the document into chunks
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=200,
    chunk_overlap=50,
    length_function=len,
)
splits = text_splitter.split_documents(documents)

print(f"Split the document into {len(splits)} chunks")


Split the document into 6 chunks


## 2. Creating Vector Embeddings and Setting up Vector Store

Now we'll create embeddings for our document chunks and store them in a vector store. We'll use OpenAI's embeddings and Chroma as our vector store.


In [5]:
# Initialize the embeddings
embeddings = OpenAIEmbeddings()

# Create a vector store
vectorstore = Chroma.from_documents(
    documents=splits,
    embedding=embeddings,
    persist_directory="chroma_db"
)

# Persist the vector store
vectorstore.persist()


/var/folders/8f/dgssdpz9545_n281yydw509h0000gn/T/ipykernel_36778/2896341674.py:12: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  vectorstore.persist()


## 3. Setting up the Retrieval Chain

Now we'll create a retrieval chain that will:
1. Take a question
2. Find relevant documents using similarity search
3. Use the retrieved documents as context to generate an answer


In [6]:
# Create a custom prompt template
prompt_template = """Use the following pieces of context to answer the question at the end. 
If you don't know the answer, just say that you don't know, don't try to make up an answer.

Context: {context}

Question: {question}

Answer:"""

PROMPT = PromptTemplate(
    template=prompt_template,
    input_variables=["context", "question"]
)

# Initialize the LLM
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)

# Create the retrieval chain
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=vectorstore.as_retriever(search_kwargs={"k": 2}),
    chain_type_kwargs={"prompt": PROMPT},
    return_source_documents=True
)


## 4. Testing the RAG System

Let's test our RAG system with some questions about the content in our document.


In [7]:
# Function to ask questions and display results
def ask_question(question):
    result = qa_chain({"query": question})
    print(f"\nQuestion: {question}")
    print(f"\nAnswer: {result['result']}")
    print("\nSource Documents:")
    for i, doc in enumerate(result["source_documents"]):
        print(f"\nDocument {i+1}:")
        print(doc.page_content)

# Test with some questions
questions = [
    "What is machine learning and how does it relate to AI?",
    "What are the applications of Computer Vision?",
    "What is NLP used for?"
]

for question in questions:
    ask_question(question)
    print("\n" + "="*80 + "\n")


/var/folders/8f/dgssdpz9545_n281yydw509h0000gn/T/ipykernel_36778/1887299745.py:3: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result = qa_chain({"query": question})



Question: What is machine learning and how does it relate to AI?

Answer: Machine learning is a subset of artificial intelligence that enables computers to learn from data without being explicitly programmed. It relates to AI by providing the technology and algorithms that allow machines to learn and improve their performance on tasks without human intervention.

Source Documents:

Document 1:
Artificial Intelligence (AI) is revolutionizing various industries. Machine learning, a subset of AI,

Document 2:
enables computers to learn from data without being explicitly programmed. Deep learning, a type of 
machine learning, uses neural networks with multiple layers to process complex patterns.



Question: What are the applications of Computer Vision?

Answer: The applications of Computer Vision include facial recognition, autonomous vehicles, and medical imaging.

Source Documents:

Document 1:
Computer Vision is the field of AI that enables computers to understand and process visual i

## Cleanup

Let's clean up our temporary files and database.


In [8]:
import shutil
import os

# Remove the sample document
if os.path.exists("sample_doc.txt"):
    os.remove("sample_doc.txt")

# Remove the Chroma database
if os.path.exists("chroma_db"):
    shutil.rmtree("chroma_db")

print("Cleanup completed!")


Cleanup completed!
